In [5]:
import pandas as pd
import sqlite3
import random
from contextlib import contextmanager

# Step 1: Generate sample weather data (similar to previous request)
def generate_sample_data(num_records=1000):
    years = list(range(1900, 2021))
    data = {
        'record_id': range(1, num_records + 1),
        'year': [random.choice(years) for _ in range(num_records)],
        'temperature_c': [random.uniform(-5, 50) for _ in range(num_records)]
    }
    return pd.DataFrame(data)

# Step 2: Simulate Hive database and table in SQLite
@contextmanager
def sqlite_connection(db_name):
    conn = sqlite3.connect(db_name)
    try:
        yield conn
    finally:
        conn.close()

def setup_hive_like_db():
    # Create a SQLite database (simulating Hive database)
    db_name = 'weather_hive1.db'
    df = generate_sample_data(1000)

    with sqlite_connection(db_name) as conn:
        # Create table (simulating Hive CREATE TABLE)
        df.to_sql('weather_data', conn, if_exists='replace', index=False)

        # Create index on year (simulating Hive CREATE INDEX)
        conn.execute('CREATE INDEX idx_year ON weather_data(year)')

        # Create view (simulating Hive CREATE VIEW)
        conn.execute('''
            CREATE VIEW positive_temps AS
            SELECT record_id, year, temperature_c
            FROM weather_data
            WHERE temperature_c > 0
        ''')

    print(f"Database '{db_name}', table 'weather_data', index 'idx_year', and view 'positive_temps' created.")

# Step 3: User-Defined Function (UDF) to convert Celsius to Fahrenheit
def celsius_to_fahrenheit(temp_c):
    return (temp_c * 9/5) + 32

# Register UDF in SQLite
def register_udf(conn):
    conn.create_function('c_to_f', 1, celsius_to_fahrenheit)
    print("UDF 'c_to_f' registered.")

# Step 4: Generate weather report (min/max temps per year)
def generate_weather_report():
    db_name = 'weather_hive.db'
    with sqlite_connection(db_name) as conn:
        # Register UDF
        register_udf(conn)

        # Query table for min/max temps
        query_table = '''
            SELECT year,
                   MIN(temperature_c) AS min_temp_c,
                   MAX(temperature_c) AS max_temp_c
            FROM weather_data
            GROUP BY year
            ORDER BY year
        '''
        report_df = pd.read_sql_query(query_table, conn)

        # Query view with UDF for max temp in Fahrenheit
        query_view = '''
            SELECT year,
                   c_to_f(MAX(temperature_c)) AS max_temp_f
            FROM positive_temps
            GROUP BY year
            ORDER BY year
        '''
        view_df = pd.read_sql_query(query_view, conn)

        # Merge results
        result = report_df.merge(view_df, on='year', how='left')
        result['max_temp_f'] = result['max_temp_f'].round(1)
        result['min_temp_c'] = result['min_temp_c'].round(1)
        result['max_temp_c'] = result['max_temp_c'].round(1)

    return result

# Step 5: Run the POC
if __name__ == "__main__":
    print("Setting up Hive-like environment...")
    setup_hive_like_db()

    print("\nGenerating Weather Temperature Statistics Report...")
    report = generate_weather_report()

    print("\n=== Weather Report ===")
    print("Year\tMin Temp (°C)\tMax Temp (°C)\tMax Temp (°F)")
    print("-" * 50)
    for _, row in report.iterrows():
        print(f"{int(row['year'])}\t{row['min_temp_c']}\t\t{row['max_temp_c']}\t\t{row['max_temp_f']}")

    print("\nSample data from view (first 5 rows):")
    with sqlite_connection('weather_hive.db') as conn:
        sample_view = pd.read_sql_query('SELECT * FROM positive_temps LIMIT 5', conn)
        print(sample_view)

Setting up Hive-like environment...
Database 'weather_hive1.db', table 'weather_data', index 'idx_year', and view 'positive_temps' created.

Generating Weather Temperature Statistics Report...
UDF 'c_to_f' registered.

=== Weather Report ===
Year	Min Temp (°C)	Max Temp (°C)	Max Temp (°F)
--------------------------------------------------
1900	-38.0		49.0		120.2
1901	-41.1		29.7		85.4
1902	-35.7		46.6		116.0
1903	-33.1		49.7		121.4
1904	-42.1		49.4		120.9
1905	-40.2		45.3		113.6
1906	-48.2		46.7		116.1
1907	-46.0		43.8		110.8
1908	-47.9		36.9		98.4
1909	-27.8		49.4		120.9
1910	-48.5		38.0		100.3
1911	-34.9		32.6		90.7
1912	-42.2		44.2		111.5
1913	-47.1		45.5		113.9
1914	3.7		43.1		109.6
1915	-36.2		49.5		121.0
1916	-26.1		48.5		119.3
1917	-42.4		16.4		61.5
1918	-48.3		39.2		102.5
1919	-45.1		40.3		104.6
1920	-48.8		36.8		98.2
1921	-43.9		21.7		71.1
1922	-36.3		48.7		119.7
1923	-20.4		48.3		119.0
1924	-21.9		46.1		115.0
1925	-49.6		49.1		120.4
1926	-34.3		40.5		104.8
1927	-25.5		30.1		86